[link](https://leportella.com/pt-br/2017/11/30/brincando-de-nlp-com-spacy.html)

> #### Instalação
    
O spaCy pode ser facilmente instalado com o pip

In [28]:
# Instalando a biblioteca
# pip install spacy

#### baixando o modelo em portugues

In [56]:
# !python -m spacy download pt
# !python -m spacy download en

> #### Usando um modelo pré-pronto

Podemos usar direto o modelo de português que acabamos de baixar. Para isso, separamos o modelo e chamamos ele genericamente de nlp. Como estamos carregando um modelo já pronto, essa declaração tende a demorar um pouquinho.

In [15]:
# Importando a biblioteca
import spacy

In [16]:
nlp = spacy.load('pt')

In [18]:
nlp

A partir de agora podemos usar esse modelo para entender frases em português. Vamos usar o nlp para estudar uma frase simples:

In [21]:
doc = nlp(u'Você encontrou o livro que eu te falei, Carla?')

**Obs:** você deve declarar a string como unicode para que ele funcione corretamente.

> ### Um pouco sobre docs e tokens…

Um Doc, objeto como aquele que acabamos de criar, é uma sequência de objetos do tipo Token e possui diversas informações sobre o texto que ele contém. Por dividir a frase em tokens, esse documento é uma estrutura iterável e portanto, deve ser acessada como tal. Já um Token é uma parte da estrutura e pode ser uma frase, palavra, uma pontuação, um espaço em branco, etc. No nosso caso, como iremos avaliar uma frase, os tokens serão constituídos de palavras e pontuações.

*Primeiro, vamos analisar a frase da maneira mais simples: dividindo-a com o método split de qualquer string.*

In [23]:
doc.text.split()

['Você', 'encontrou', 'o', 'livro', 'que', 'eu', 'te', 'falei,', 'Carla?']

Podemos ver que apesar de ser coerente a divisão por espaços, o verbo falei e a vírgula estão dentro de um mesmo `token`, assim como Carla e a interrogação. O nlp consegue entender a diferença entre eles e, portanto, quando usamos os `tokens` dentro da estrutura do documento, temos uma divisão mais coerente:

In [24]:
[token for token in doc]

[Você, encontrou, o, livro, que, eu, te, falei, ,, Carla, ?]

Repare que agora a estrutura considerou a pontução e as palavras como estruturas separadas. Também não temos mais uma **lista de strings**, mas uma **lista de Tokens**.

Se não quisermos os objetos Tokens, mas sim as strings que cada Token contém podemos usar o método `.orth_`:

In [26]:
[token.orth_ for token in doc]

['Você',
 'encontrou',
 'o',
 'livro',
 'que',
 'eu',
 'te',
 'falei',
 ',',
 'Carla',
 '?']

> ### Entendendo diferença entre palavras e pontuações

Como o spaCy entende que existe uma diferença entre uma **palavra e uma pontuação**, também podemos fazer filtragens. E se eu quisesse apenas as palavras da frase?

In [27]:
[token.orth_ for token in doc if not token.is_punct]

['Você', 'encontrou', 'o', 'livro', 'que', 'eu', 'te', 'falei', 'Carla']

> ### Similaridade

O spaCy também permite avaliar similaridade entre palavras. O método `.similarity` de um Token avalia a ***similaridade semântica estimada entre as palavras. Quanto maior o valor, mais similar são as palavras.***

Vamos avaliar a similaridade entre 3 palavras: `você, livro e eu`.

Primeiro vamos armazenar o tokens em uma lista para acessá-los de forma independente:

In [31]:
tokens = [token for token in doc]

Dessa forma, temos que `tokens[0]` representa o meu token da palavra **Você** e `tokens[5]` representa a palavra **eu**. 

A análise de similaridade pode ser dada por:

In [36]:
tokens[0].similarity(tokens[5])

/home/dieinimy/.pyenv/versions/3.7.4/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)


0.4892635

Legal, temos um valor de **0,4892635**. E o que isso significa? Sabemos intuitivamente que eu e você devem ser muito mais similares que você e livro, por exemplo. Vamos investigar:



In [39]:
tokens[0].similarity(tokens[3])

/home/dieinimy/.pyenv/versions/3.7.4/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)


-0.0045953672

O valor é negativo, ou seja, você de fato está semânticamente muito mais próximo de eu do que de livro, o que faz todo sentido, certo?

> ### Análise de classes gramaticais

Podemos também entender as classes gramaticais de cada palavra dentro do nosso contexto:

In [40]:
[(token.orth_, token.pos_) for token in doc]

[('Você', 'PRON'),
 ('encontrou', 'VERB'),
 ('o', 'DET'),
 ('livro', 'NOUN'),
 ('que', 'PRON'),
 ('eu', 'PRON'),
 ('te', 'PRON'),
 ('falei', 'NOUN'),
 (',', 'PUNCT'),
 ('Carla', 'PROPN'),
 ('?', 'PUNCT')]

Então assim conseguimos ver que encontrou e falei são os verbos da frase. E a vírgula e o ponto de interrogação foi corretamente definido como pontuação (PUNCT).

> ### Encontrei, encontraram, encontrarão, encontrariam….


Agora, imagine que você tem um texto enorme e diversos tempos verbais diferentes. A análise passa a ser infinitamente mais complicada! O que podemos fazer, então, é analisar não o verbo no tempo verbal que ele foi escrito, mas ele em sua raiz. O nome desse método de encontrar **a raiz das palavras é lematização** por isso, o método `.lemma_` faz exatamente isso. Vamos olhar:

In [43]:
[token.lemma_ for token in doc if token.pos_ == 'VERB']

['encontrar']

E isso vale para diversos tempos verbais **MESMO!**

In [47]:
doc = nlp(u'encontrar encontrei')
tokens = [token for token in doc]
tokens[0].is_ancestor(tokens[1])

True

> ### E por fim… entidades
Por fim, podemos avaliar as entidades presentes em uma frase. Por exemplo, peguemos a frase:

In [48]:
doc = nlp(u'Machado de Assis um dos melhores escritores do Brasil, foi o primeiro presidente da Academia Brasileira de Letras')

Se formos analisar as entidades presentes nessa frase, percebemos que temos 3 entidades identificadas automaticamente:

In [49]:
doc.ents

(Machado de Assis, Brasil, Academia Brasileira de Letras)

Ao analisarmos detalhadamente, podemos ver que o Spacy identificou Machado de Assis como uma pessoa (PER de person, em inglês), Brasil como um local (LOC) e Academia Brasileira de Letras como uma organização (ORG).

In [50]:
[(entity, entity.label_) for entity in doc.ents]

[(Machado de Assis, 'PER'),
 (Brasil, 'LOC'),
 (Academia Brasileira de Letras, 'ORG')]

Claro que em inglês o modelo está bem avançado, e consegue identificar entidades bem mais complexas do que eu pude verificar nos textos em português. Por exemplo:

In [51]:
wiki_obama = """Barack Obama is an American politician who served as
     the 44th President of the United States from 2009 to 2017. He is the first
     African American to have served as president,
     as well as the first born outside the contiguous United States."""

In [58]:
nlp = spacy.load('en')
nlp_obama = nlp(wiki_obama)
[(i, i.label_) for i in nlp_obama.ents]

[(Barack Obama, 'PERSON'),
 (American, 'NORP'),
 (44th, 'ORDINAL'),
 (the United States, 'GPE'),
 (2009, 'DATE'),
 (2017, 'DATE'),
 (first, 'ORDINAL'),
 (African American, 'NORP'),
 (first, 'ORDINAL'),
 (United States, 'GPE')]